In [2]:
import pandas as pd
import numpy as np
import random

# Grid

In [5]:
grid = np.zeros((2,5,5))
grid

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

# Pieces

In [148]:
# Player 1
p1_4x1 = np.array([[[1,1,1,1]], [[0,0,0,0]]])
p1_3x1 = np.array([[[1,1,1]], [[0,0,0]]])
p1_2x1 = np.array([[[1,1]], [[0,0]]])
p1_L = np.array([[[1,0,0], [1,1,1]], [[0,0,0],[0,0,0]]])
p1_square = np.array([[[1,1],[1,1]], [[0,0],[0,0]]])
p1_corner = np.array([[[1, 0],[1, 1]], [[0, 0],[0, 0]]])
p1_pipe = np.array([[[0,1,0],[1,1,1]], [[0,0,0],[0,0,0]]])
p1_bend = np.array([[[0,1,1],[1,1,0]], [[0,0,0],[0,0,0]]])

p1_pieces = [
    p1_4x1,
    p1_3x1,
    p1_2x1,
    p1_L,
    p1_square,
    p1_corner,
    p1_pipe,
    p1_bend
]

# Player 2
p2_4x1 = np.array([[[2,2,2,2]], [[0,0,0,0]]])
p2_3x1 = np.array([[[2,2,2]], [[0,0,0]]])
p2_2x1 = np.array([[[2,2]], [[0,0]]])
p2_L = np.array([[[2,0,0], [2,2,2]], [[0,0,0],[0,0,0]]])
p2_square = np.array([[[2,2],[2,2]], [[0,0],[0,0]]])
p2_corner = np.array([[[2,0],[2,2]], [[0,0],[0,0]]])
p2_pipe = np.array([[[0,2,0],[2,2,2]], [[0,0,0],[0,0,0]]])
p2_bend = np.array([[[0,2,2],[2,2,0]], [[0,0,0],[0,0,0]]])

p2_pieces = [
    p2_4x1,
    p2_3x1,
    p2_2x1,
    p2_L,
    p2_square,
    p2_corner,
    p2_pipe,
    p2_bend
]

# Play

In [69]:
t = np.array([[[1,1,1]]])
t

array([[[1, 1, 1]]])

In [126]:
# t = np.array([[[1, 0],[1, 1]], [[0, 0],[0, 0]]])
t = np.array([[[1,1,1]], [[0,0,0]]])
k1 = random.randint(0,3)
k2 = random.randint(0,3)

np.rot90(np.rot90(t, k=k1, axes=(0,1)), k=k2, axes=(1,2))

array([[[1],
        [1],
        [1]],

       [[0],
        [0],
        [0]]])

In [113]:
t = np.array([[1,1,1]])
k1 = random.randint(0,3)
np.rot90(t, k=k1, axes=(0,1))

array([[1, 1, 1]])

In [147]:
t = np.array([[[1, 0],[1, 1]], [[0, 0],[0, 0]]])
fill_out(orient(t))

array([[[0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]])

In [152]:
def orient(p):
    k1 = random.randint(0,3)
    k2 = random.randint(0,3)
    return np.rot90(np.rot90(p, k=k1, axes=(0,1)), k=k2, axes=(1,2))


def fill_out(p):
    stacks, rows, cols = p.shape

    stack_max_start = 2-stacks
    row_max_start = 5-rows
    col_max_start = 5-cols

    stack_start = random.randint(0, stack_max_start)
    row_start = random.randint(0, row_max_start)
    col_start = random.randint(0, col_max_start)
    
    stack_top_pad = stack_start
    stack_bottom_pad = stack_max_start - stack_start
    row_top_pad = row_start
    row_bottom_pad = row_max_start - row_start
    col_left_pad = col_start
    col_right_pad = col_max_start - col_start

    return np.pad(p, ((stack_top_pad,stack_bottom_pad), (row_top_pad,row_bottom_pad), (col_left_pad,col_right_pad)))


def check_adjacent(grid, layer, player):
    adjacent = np.zeros((5,5))
    
    for i, row in enumerate(grid):
        for j, val in enumerate(row):
            if val == player:
                # 4 is hardcoded
                if (j>0) and (grid[i, j-1] == 0):
                    adjacent[i, j-1] = player
                if (j<4)  and (grid[i, j+1]) == 0:
                    adjacent[i, j+1] = player
                if (i>0) and (grid[i-1, j]) == 0:
                    adjacent[i-1, j] = player
                if (i<4) and (grid[i+1, j]) == 0:
                    adjacent[i+1, j] = player
                    
    adjacent_bool = False if (adjacent*layer).max() == 0 else True
    return adjacent, adjacent_bool


def play_move(piece_list, grid, player, start):
    
    attempt_counter = 0
    
    while attempt_counter < 1000:
    
        # Randomly choose a piece
        piece_i = random.randint(0, len(piece_list)-1)
        piece = piece_list[piece_i]

        # Randomly rotate piece and place it's location
        layer = fill_out(orient(piece))

        # Check if it is a legal move
        
        # Are any spots already taken?
        spot_bool = False
        if (grid*layer).max() == 0:
            spot_bool = True

#         # Check if adjacent to an existing piece (unless it's the first turn)
#         if (spot_bool is True) and (start is False):
#             adjacent, adjacent_bool = check_adjacent(grid, layer, player)
#         else:
#             adjacent_bool = True
#             adjacent = ''
        
        # Make that play if it is legal
        if (spot_bool is True):# and (adjacent_bool is True):
            grid+=layer
            del piece_list[piece_i]
            # print('Adjacent:\n', adjacent, '\n')
            print(f'# of attempts: {attempt_counter}')
            print(grid, '\n')
            return grid, piece_list
        
        # Keep track of attempts
        attempt_counter+=1
        if attempt_counter==1000:
            print('Maxed out attempts\n')
            return grid, piece_list

In [153]:
i=0
grid = np.zeros((2,5,5))
p1_pieces = [p1_4x1, p1_3x1, p1_2x1, p1_L, p1_square, p1_corner, p1_pipe, p1_bend]
p2_pieces = [p2_4x1, p2_3x1, p2_2x1, p2_L, p2_square, p2_corner, p2_pipe, p2_bend]

while i<=20:
    # Figure out whose turn it is and get their available pieces
    piece_list = p1_pieces if i%2 == 0 else p2_pieces
    player = 1 if i%2 == 0 else 2
    
    # Indicate if it's the player's first turn
    start = True if i<=1 else False
    
    # Play a piece
    if len(piece_list) > 0:
        grid, piece_list = play_move(piece_list, grid, player, start)
        
    i+=1

# of attempts: 0
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]] 

# of attempts: 1
[[[0. 0. 0. 0. 2.]
  [0. 0. 0. 0. 2.]
  [0. 0. 0. 0. 2.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 2.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]] 

# of attempts: 1
[[[0. 0. 0. 0. 2.]
  [0. 0. 0. 0. 2.]
  [0. 0. 0. 0. 2.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 1.]]

 [[0. 0. 0. 0. 2.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]] 

# of attempts: 0
[[[0. 0. 0. 2. 2.]
  [0. 0. 0. 2. 2.]
  [0. 0. 0. 2. 2.]
  [0. 0. 0. 2. 0.]
  [0. 0. 1. 1. 1.]]

 [[0. 0. 0. 0. 2.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]] 

# of attempts: 1
[[[0. 0. 0. 2. 2.]
  [0. 1. 0. 2. 2.]
  [0. 1. 0. 2. 2.]
  [0. 1. 0. 2. 0.]
  [0. 0. 1. 1. 1.]]

 [[0. 0. 0. 0. 2.]
  [0. 0. 0. 1. 

In [51]:
grid

array([[0., 2., 1., 1., 1.],
       [1., 2., 1., 1., 2.],
       [1., 1., 1., 0., 2.],
       [1., 1., 2., 2., 2.],
       [1., 1., 2., 2., 2.]])

In [53]:
t = np.array([[0,1,0,0,0], [0,1,0,0,0], [0,1,1,0,0], [0,0,0,0,0], [0,0,0,0,0]])
t

array([[0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [54]:
t[0]

array([0, 1, 0, 0, 0])

In [76]:
adjacent = np.zeros((5,5))

for i, row in enumerate(t):
    for j, val in enumerate(row):
        if val == 1:
            # 4 is hardcoded
            if (j>0) and (t[i, j-1] == 0):
                adjacent[i, j-1] = 1
            if (j<4)  and (t[i, j+1]) == 0:
                adjacent[i, j+1] = 1
            if (i>0) and (t[i-1, j]) == 0:
                adjacent[i-1, j] = 1
            if (i<4) and (t[i+1, j]) == 0:
                adjacent[i+1, j] = 1

In [77]:
adjacent

array([[1., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0.],
       [0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [90]:
grid.max()

2.0

In [31]:
i=0
grid = np.zeros((5,5))
possible_moves = []

while i<=5:
    # Figure out whose turn it is and get their available pieces
    piece_list = p1_pieces if i%2 == 0 else p2_pieces
    player = 1 if i%2 == 0 else 2
    
    # Randomly choose a piece
    piece_i = random.randint(0, len(piece_list)-1)
    piece = piece_list[piece_i]
    
    #print(piece)
    
    # Randomly rotate piece and place it's location
    layer = fill_out(orient(piece))
    
    #print(layer, '\n')
    
    # Check if it is a legal move
    # Are any spots already taken?
    if (grid*layer).max() == 0:
        grid+=layer
        
    print(grid, '\n')
    
    i+=1

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0.]] 

[[0. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0.]
 [2. 2. 1. 0. 0.]
 [2. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0.]] 

[[0. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0.]
 [2. 2. 1. 0. 0.]
 [2. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0.]] 

[[0. 0. 0. 0. 0.]
 [2. 0. 0. 2. 2.]
 [2. 2. 1. 2. 2.]
 [2. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0.]] 

[[0. 0. 0. 0. 0.]
 [2. 0. 0. 2. 2.]
 [2. 2. 1. 2. 2.]
 [2. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0.]] 

[[0. 0. 0. 0. 0.]
 [2. 0. 0. 2. 2.]
 [2. 2. 1. 2. 2.]
 [2. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0.]] 



In [27]:
grid

array([[0., 0., 1., 1., 1.],
       [0., 0., 0., 2., 2.],
       [0., 0., 1., 2., 2.],
       [0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0.]])

In [70]:
grid = np.array([[0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]])
grid

array([[0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [71]:
p = np.array([[0, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]])
p

array([[0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [75]:
res = grid+p
res

array([[0, 1, 0, 0, 0],
       [0, 1, 2, 0, 0],
       [0, 0, 2, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [76]:
res.max()

2

In [83]:
set(list(res.flatten()))

{0, 1, 2}

In [84]:
res

array([[0, 1, 0, 0, 0],
       [0, 1, 2, 0, 0],
       [0, 0, 2, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [18]:
{0,1,2,3}.intersection({2})

{2}

In [172]:
def orient(p):
    k = random.randint(1,4)
    return np.rot90(p, k=k)


def fill_out(p):
    rows, cols = p.shape

    row_max_start = 5-rows
    col_max_start = 5-cols

    row_start = random.randint(0, row_max_start)
    col_start = random.randint(0, col_max_start)

    row_top_pad = row_start
    row_bottom_pad = row_max_start - row_start
    col_left_pad = col_start
    col_right_pad = col_max_start - col_start

    return np.pad(p, ((row_top_pad,row_bottom_pad), (col_left_pad,col_right_pad)))

In [181]:
fill_out(orient(p1_L))

array([[0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [162]:
orient(p1_L)

array([[0, 1],
       [0, 1],
       [1, 1]])

In [142]:
rows, cols = p1_L.shape

row_max_start = 5-rows
col_max_start = 5-cols

row_start = random.randint(0, row_max_start)
col_start = random.randint(0, col_max_start)

row_top_pad = row_start
row_bottom_pad = row_max_start - row_start
col_left_pad = col_start
col_right_pad = col_max_start - col_start

np.pad(p1_L, ((row_top_pad,row_bottom_pad), (col_left_pad,col_right_pad)))

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0]])

# Moves

In [12]:
np.rot90(p2, k=2)

array([[1, 1, 1],
       [0, 0, 1]])